# ДЗ4. Классификация новостей
Будем собирать новости и классифицировать их по рубрикам с помощью МО.

## Сбор данных (5 баллов)

Нужно собрать данные из пяти рубрик сайта lenta.ru. Этот пункт можно пропустить и перейти ко второй части, за которую можно получить 7 баллов.

1. Ссылки на рубрики даны ниже. 
   
       1.1 На каждой из этих страниц новости рубрики располагаются в блоке, размеченном тегом `<<div class="item news b-tabloid__topic_news">`.
       1.2 Далее внутри этого блока нужно найти все ссылки (тег `<a>`) и забрать из них значение атрибута (`href`).
       1.3 Каждую ссылку нужно восстановить до полного вида, добавив вначало строку `'https://lenta.ru'`
    
2. После того как для рубрики собраны все ссылки, нужно с каждой из них собрать текст новости.
    
        2.1 Новость располагается в блоке, размеченном тегом `<div class='b-topic__content'>`.
        2.2 Текст новости разбит на параграфы (тег `<p>`), особенностью которых является то, что у них нет класса (`class=None`)
        2.3 Текст новости нужно собрать в единую строку. Например, собрав все параграфы в список, а потом объединить их через `.join()` и пустую строку.

3. Каждую новость нужно сохранить вместе с лейблом — названием рубрики, к которой она принадлежит. Лейбл проще всего достать из ссылки рубрики, разбив ее с помощью `.split()` по слэшу.

В итоге у вас должен получиться датафрейм pandas со всеми собранными новстями (~350 штук), в котором есть две колонки — `'text'` и '`label'`. Удобнее всего его получить написав фукнцию, которая работает следующим образом:

    1. Функция принимает в качестве аргумента ссылку на рубрику.
    2. Из ссылки извлекается название рубрики.
    3. Создается объект BeautifulSoup с исходным текстом страницы рубрики.
    4. В объекте BS находится нужный блок с новостями рубрики (см. п. 1.1).
    5. Внутри этого блока находятся все ссылки на статьи рубрики (п. 1.2 и 1.3).
    6. Создается список, в котором будут храниться новости рубрики.
    7. Затем для каждой ссылки рубрики выполняются следующие операции:
        1) Создается объект BS из исходного кода страницы новости.
        2) Внутри объекта BS ищется блок с текстом новости (п. 2.1).
        3) Внутри этого блока ищутся все параграфы текста, которые объединяются в единую строку (п. 2.2 и 2.3)
        4) В список из пункта 6 этой инструкции добавляется кортеж или список из двух строк — текст новости и ее рубрика.
        5) После того как функция прошлась по всем новостям рубрики, она возвращает список из пункта 6 со всеми новостями рубрики.
    8. Функция запускается для всех рубрик. Результаты работы функции (списки кортежей) добавляются в новый список через `+=` или метод `.extend()`.

4. Получившийся список преобразуется в объект pandas.DataFrame с колонками `'text'` и `'label'`. Итоговый датафрейм сохраняется в файл с расширением '.csv'
    

In [1]:
rubrics = ['https://lenta.ru/rubrics/science/', 'https://lenta.ru/rubrics/culture/',
           'https://lenta.ru/rubrics/sport/', 'https://lenta.ru/rubrics/economics/',
           'https://lenta.ru/rubrics/travel/']

## Нормализация текста (3 балла)

Если вы не собрали самостоятельно данные, то можно воспользоваться датасетом по ссылке.

1. Вам нужно написать функцию, которая будет нормализовать текст — очистит его от знаков препинания и приведет все слова к словарной форме.
    
        1.1 Внутри функции используйте шаблон регулярных выражений, чтобы оставить в тексте только слова, написанные кириллицей. Слова с написанием через дефис должны оставаться одним токеном (например, 'из-за'). Например, можно получить список всех токенов из новости с помощью `re.find_all()`.
        1.2 Каждое слово нужно привести к словарной форме. Например, это можно сделать методом normal_forms класса MorphAnalyzer из библиотеки pymorphy2. Первая форма в этом списке — наиболее вероятная.
        
Ваша функция должна принимать на вход строку, а возвращать ее нормализованный вариант.

2. Примените вашу функцию к колонке 'text' и сохраните отнормализованный текст в новой колонке датафрейма.


In [44]:
import pandas as pd

In [54]:
df = pd.DataFrame(all_news, columns=['text', 'label'])
df.head()

,text,label
0,Ученые Университета Торонто в Канаде определил...,science
1,"Международная группа ученых раскрыла загадку, ...",science
2,«Роскосмос» намерен выделить на первый этап из...,science
3,Специалисты Потсдамского института изменения к...,science
4,Microsoft отказалась от выпуска операционной с...,science


In [143]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rogovich/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [125]:
import pymorphy2
import re
lemm = pymorphy2.MorphAnalyzer()

def normalize_text(text):
    text = re.findall(r'\b[А-яЕё]+(?:-[А-яЕё]+)*\b', text)
    return ' '.join([lemm.normal_forms(word)[0] for word in text])
    
df['text'] = df['text'].apply(normalize_text)

In [182]:
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(df['text'], df['label'])

In [179]:
model = CountVectorizer(max_df=0.8)

In [185]:
train_features = model.fit_transform(X_tr)
test_features = model.transform(X_te)

In [186]:
from sklearn.ensemble import RandomForestClassifier

In [187]:
forest = RandomForestClassifier()
forest.fit(train_features, y_tr)
forest.predict(test_features)

array(['science', 'sport', 'science', 'economics', 'culture', 'economics',
       'economics', 'science', 'sport', 'science', 'science', 'science',
       'economics', 'economics', 'economics', 'travel', 'science',
       'sport', 'sport', 'science', 'science', 'economics', 'travel',
       'culture', 'travel', 'culture', 'science', 'culture', 'science',
       'science', 'economics', 'science', 'economics', 'travel',
       'science', 'culture', 'travel', 'culture', 'travel', 'culture',
       'travel', 'economics', 'culture', 'travel', 'culture', 'sport',
       'science', 'culture', 'economics', 'culture', 'science', 'culture',
       'economics', 'economics', 'science', 'sport', 'economics',
       'culture', 'science', 'sport', 'sport', 'science', 'science',
       'economics', 'sport', 'economics', 'culture', 'travel',
       'economics', 'travel', 'economics', 'economics', 'sport',
       'culture', 'science', 'science', 'economics', 'travel', 'travel',
       'culture', 'cultur

In [190]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(forest.predict(test_features), y_te)

0.9222222222222223

In [199]:
pd.DataFrame(confusion_matrix(y_te, forest.predict(test_features), 
            labels=sorted(set(y_te))), columns = sorted(set(y_te)), 
            index=sorted(set(y_te)))

,culture,economics,science,sport,travel
culture,18,0,0,0,0
economics,0,18,3,0,0
science,0,2,21,0,0
sport,1,0,0,10,0
travel,0,0,1,0,16


In [202]:
list(y_te).count('sport')

11

In [204]:
list(forest.predict(test_features)).count('culture')

19

In [211]:
for i in range(len(y_te)):
    if list(y_te)[i] != list(forest.predict(test_features))[i]:
        print(i, list(y_te)[i], list(forest.predict(test_features))[i])

14 science economics
16 economics science
29 economics science
35 sport culture
48 science economics
82 travel science
89 economics science


In [3]:
X_te.iloc[89]

NameError: name 'X_te' is not defined

In [1]:
df

NameError: name 'df' is not defined